In [4]:
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# 1. Sweep 구성 설정 (Random 방식 사용)
sweep_configuration = {
    'method': 'random',  # Random search로 변경
    'name': 'jellyfish_sweep',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},  # 최적화할 지표: val_accuracy
    'parameters': {
        'lr': {  # 학습률: log_uniform 분포 사용
            'min': 1e-5,
            'max': 1e-2,
            'distribution': 'log_uniform'
        },
        'epochs': {  # 에폭 수
            'values': [5, 10, 20]
        },
        'batch_size': {  # 배치 크기
            'values': [16, 32]
        }
    }
}

# 2. 모델 정의 함수
def create_model(lr):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(6, activation='softmax')  # 6종의 해파리를 분류
    ])
    
    model.compile(optimizer=Adam(learning_rate=lr), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# 3. 데이터 로딩 및 전처리
def load_data(train_dir, valid_dir):
    # 데이터 증강을 위한 ImageDataGenerator 설정
    train_datagen = ImageDataGenerator(rescale=1./255, 
                                       horizontal_flip=True, 
                                       rotation_range=30)
    valid_datagen = ImageDataGenerator(rescale=1./255)

    # 학습 및 검증 데이터 로딩
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
    
    valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
    
    return train_generator, valid_generator

# 4. 학습 함수 정의
def train():
    # W&B 설정
    wandb.init(project='jellyfish_classification', config=sweep_configuration['parameters'])
    config = wandb.config

    # 데이터 로딩
    train_generator, valid_generator = load_data('Train_Test_Valid/Train', 'Train_Test_Valid/valid')
    
    # 모델 생성
    model = create_model(config.lr)
    
    # 모델 학습
    history = model.fit(
        train_generator,
        epochs=config.epochs,
        validation_data=valid_generator,
        batch_size=config.batch_size,
        callbacks=[WandbCallback()]
    )
    
    # 성능 기록
    wandb.log({"val_accuracy": history.history['val_accuracy'][-1]})

# 5. Sweep 초기화 및 실행
sweep_id = wandb.sweep(sweep=sweep_configuration, project='jellyfish_classification')
wandb.agent(sweep_id, function=train, count=10)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: rpl6i6uu
Sweep URL: https://wandb.ai/kyskyj22-aiffel/jellyfish_classification/sweeps/rpl6i6uu


wandb: Agent Starting Run: 2q21nu13 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 1.003450881832327
wandb: Currently logged in as: kyskyj22 (kyskyj22-aiffel). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Found 900 images belonging to 6 classes.
Found 39 images belonging to 6 classes.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


...

wandb: Adding directory to artifact (/aiffel/aiffel/jellyfish/wandb/run-20241115_080738-arolssqa/files/model-best)... Done. 3.5s


Epoch 5/20
29/29 [==============================] - 10s 335ms/step - loss: 1.8338 - accuracy: 0.1522 - val_loss: 1.8021 - val_accuracy: 0.1795
INFO:tensorflow:Assets written to: /aiffel/aiffel/jellyfish/wandb/run-20241115_080738-arolssqa/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/jellyfish/wandb/run-20241115_080738-arolssqa/files/model-best)... Done. 3.2s


Epoch 6/20
29/29 [==============================] - 10s 340ms/step - loss: 1.8405 - accuracy: 0.1489 - val_loss: 1.8335 - val_accuracy: 0.1795
Epoch 7/20
29/29 [==============================] - 9s 324ms/step - loss: 1.8359 - accuracy: 0.1378 - val_loss: 1.8382 - val_accuracy: 0.1795
Epoch 8/20
29/29 [==============================] - 9s 322ms/step - loss: 1.8512 - accuracy: 0.1622 - val_loss: 1.8664 - val_accuracy: 0.1795
Epoch 9/20
29/29 [==============================] - 10s 326ms/step - loss: 1.8282 - accuracy: 0.1522 - val_loss: 1.8223 - val_accuracy: 0.1795
Epoch 10/20
29/29 [==============================] - 10s 327ms/step - loss: 1.8247 - accuracy: 0.1622 - val_loss: 1.8337 - val_accuracy: 0.1538
Epoch 11/20
29/29 [==============================] - 10s 327ms/step - loss: 1.8431 - accuracy: 0.1700 - val_loss: 1.8083 - val_accuracy: 0.1538
Epoch 12/20
29/29 [==============================] - 9s 321ms/step - loss: 1.8328 - accuracy: 0.1589 - val_loss: 1.8038 - val_accuracy: 0.1795

accuracy,▆▅▆▃▃▃▁▅▃▅▆▄▆▃▄▆▆▄█▄
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,███▅█████▅▅█▁▅█▅█████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.16111
best_epoch,4
best_val_loss,1.80208
epoch,19
loss,1.83934
val_accuracy,0.17949


In [3]:
train_dir = 'Train_Test_Valid/Train' 
valid_dir = 'Train_Test_Valid/valid'
test_dir = 'Train_Test_Valid/test'